In [1]:
import numpy as np
import pandas as pd
import random
from matplotlib import pyplot as plt
from numba import jit
from tqdm import tqdm
from itertools import combinations, permutations
import time

In [2]:
data_A = np.loadtxt('../data/TSPA.csv', delimiter=';').astype(np.int64)
data_B = np.loadtxt('../data/TSPB.csv', delimiter=';').astype(np.int64)
data_C = np.loadtxt('../data/TSPC.csv', delimiter=';').astype(np.int64)
data_D = np.loadtxt('../data/TSPD.csv', delimiter=';').astype(np.int64)

In [3]:
def create_cost_matrix(data):
    x = data[:, :1]
    y = data[:, 1:2]
    cost = data[:, 2:3]
    return (((x - x.reshape(1, -1))**2 + (y - y.reshape(1, -1))**2) ** (1/2) + cost.reshape(1, -1)).round().astype(np.int64)

def create_dist_matrix(data):
    x = data[:, :1]
    y = data[:, 1:2]
    #cost = data[:, 2:3]
    return (((x - x.reshape(1, -1))**2 + (y - y.reshape(1, -1))**2) ** (1/2)).round().astype(np.int64)

In [4]:
cost_matrix_A = create_cost_matrix(data_A)

In [5]:
cost_matrix_A

array([[  84, 2032, 2098, ..., 4159, 3783, 1514],
       [1633,  483, 2398, ..., 3349, 2266,  817],
       [ 720, 1419, 1462, ..., 3640, 3149,  964],
       ...,
       [2782, 2371, 3641, ..., 1461, 2908, 2554],
       [2558, 1440, 3302, ..., 3060, 1309, 1773],
       [1234,  936, 2062, ..., 3651, 2718,  364]])

In [6]:
cost_matrix_B = create_cost_matrix(data_B)

In [7]:
cost_matrix_C = create_cost_matrix(data_C)

In [8]:
cost_matrix_D = create_cost_matrix(data_D)

In [9]:
dist_matrix_A = create_dist_matrix(data_A)
dist_matrix_B = create_dist_matrix(data_B)
dist_matrix_C = create_dist_matrix(data_C)
dist_matrix_D = create_dist_matrix(data_D)

In [10]:
def plot(data, solution):
    data_ordered = np.array([data[i] for i in solution])
    all_data = np.array([data[i] for i in range(200)])

    plt.figure(figsize=(10, 10), dpi=80)

    plt.scatter(data_ordered[:,0], data_ordered[:,1], s=data_ordered[:,2]/data_ordered[:,2].max()*200, c='b')
    plt.scatter(all_data[:,0], all_data[:,1], s=all_data[:,2]/all_data[:,2].max()*200, c='b')
    plt.plot(data_ordered[:,0], data_ordered[:,1], 'y-')
    plt.plot([data_ordered[0,0], data_ordered[-1,0]], [data_ordered[0,1], data_ordered[-1,1]], 'y-')
    plt.show()

In [11]:
def calculate_performance(cycle, cost_matrix):
    total_sum = 0
    for i in range(len(cycle)-1):
        total_sum += cost_matrix[cycle[i], cycle[i+1]]
    total_sum += cost_matrix[cycle[-1], cycle[0]]
    return total_sum

In [12]:
np.arange(0, 100)

array([ 0,  1,  2,  3,  4,  5,  6,  7,  8,  9, 10, 11, 12, 13, 14, 15, 16,
       17, 18, 19, 20, 21, 22, 23, 24, 25, 26, 27, 28, 29, 30, 31, 32, 33,
       34, 35, 36, 37, 38, 39, 40, 41, 42, 43, 44, 45, 46, 47, 48, 49, 50,
       51, 52, 53, 54, 55, 56, 57, 58, 59, 60, 61, 62, 63, 64, 65, 66, 67,
       68, 69, 70, 71, 72, 73, 74, 75, 76, 77, 78, 79, 80, 81, 82, 83, 84,
       85, 86, 87, 88, 89, 90, 91, 92, 93, 94, 95, 96, 97, 98, 99])

In [13]:
@jit()
def random_solution(cost_matrix, limit=100):
    random_solution_list = np.arange(0, 100)
    np.random.shuffle(random_solution_list)
    return random_solution_list[:limit]

solution = random_solution(cost_matrix_A, 100)
print(solution)

[ 8 27 93 88 58 59 63 36 40 85 55 94 47 42 75 72 82 22 84 41 81  5 67 79
  9 49 14 56 17 44 48 21 11 50 19 78 46 33 90 18 76 87 34 97 95 86 10 31
 73 24 32 83 53 15  7 64  6 38 71 26 61 28 68 62  3  4 30 66 35 80 37 16
  1 23 92 99 96 45 51 43 57 89 98 70 69 29 12 52 91  0 25 54 39 77 74 60
 65  2 13 20]


In [14]:
# %%timeit
# random_solution(cost_matrix_A)

In [15]:
@jit()
def greedy_cycle(cost_matrix, current_id, limit=100):
    all_ids = set(list(range(0,len(cost_matrix))))
    all_ids.remove(current_id)
    solution = [current_id]
    
    for _ in range(1):
        min_val = 99999
        min_id = -1
        for next_id in all_ids:
            if cost_matrix[current_id][next_id] < min_val:
                min_val = cost_matrix[current_id][next_id]
                min_id = next_id
        solution.append(min_id)
        all_ids.remove(min_id)
        current_id = min_id
    
    while len(solution) < limit:
        min_delta = 99999
        min_id = -1
        insert_id = -1
        for i in range(len(solution)-1):
            for next_id in all_ids:
                delta = cost_matrix[solution[i]][next_id] + cost_matrix[next_id][solution[i+1]] - cost_matrix[solution[i]][solution[i+1]]
                if delta < min_delta:
                    min_delta = delta
                    min_id = next_id
                    insert_id = i
        for next_id in all_ids:
            delta = cost_matrix[solution[-1]][next_id] + cost_matrix[next_id][solution[0]] - cost_matrix[solution[-1]][solution[0]]
            if delta < min_delta:
                min_delta = delta
                min_id = next_id
                insert_id = i
        solution.insert(insert_id+1, min_id)
        all_ids.remove(min_id)

    return np.array(solution)

In [16]:
calculate_performance(greedy_cycle(cost_matrix_A, 2), cost_matrix_A)

77328

In [17]:
# %%timeit
# greedy_cycle(cost_matrix_A, 0)

In [18]:
# @jit()
def calculate_closest_neighbours(cost_matrix, n_neighbours=10):
    def extract_nodes(ll):
        return [x[1] for x in ll]
    
    diag_matrix = np.diag(range(len(cost_matrix))).astype(float)
    np.fill_diagonal(diag_matrix, np.inf)
    
    new_cost_matrix = cost_matrix + diag_matrix

    closest_neighbours = dict()
    for i in range(len(new_cost_matrix)):
        closest_neighbours[i] = extract_nodes(sorted(zip(new_cost_matrix[i], range(len(new_cost_matrix))), key=lambda x: x[0])[:n_neighbours])
    return closest_neighbours

In [19]:

def calculate_closest_neighbours_solution(cost_matrix, solution, n_neighbours=10):
    def extract_nodes(ll):
        return [solution[x[1]] for x in ll]
    
    diag_matrix = np.diag(range(len(cost_matrix))).astype(float)
    np.fill_diagonal(diag_matrix, np.inf)
    
    new_cost_matrix = cost_matrix[solution, :][:, solution] + diag_matrix[solution, :][:, solution]

    closest_neighbours = dict()
    for i in range(len(new_cost_matrix)):
        closest_neighbours[solution[i]] = extract_nodes(sorted(zip(new_cost_matrix[i], range(len(new_cost_matrix))), key=lambda x: x[0])[:n_neighbours])
    return closest_neighbours

In [20]:
solution = list(range(15)) + list(range(100, 115))

In [21]:
# %%timeit
# calculate_closest_neighbours_solution(cost_matrix_A, solution)

In [22]:
# @jit()
def calculate_change_before(enhanced_solution, cost_matrix, position_1, position_2):
    return (cost_matrix[enhanced_solution[position_1], enhanced_solution[position_2]] + cost_matrix[enhanced_solution[position_1-1], enhanced_solution[position_2-1]] -
            ( cost_matrix[enhanced_solution[position_1-1], enhanced_solution[position_1]] + cost_matrix[enhanced_solution[position_2-1], enhanced_solution[position_2]] )
            )

def calculate_change_after(enhanced_solution, cost_matrix, position_1, position_2):
    return (cost_matrix[enhanced_solution[position_1], enhanced_solution[position_2]] + cost_matrix[enhanced_solution[(position_1+1)%len(enhanced_solution)], enhanced_solution[(position_2+1)%len(enhanced_solution)]] -
            ( cost_matrix[enhanced_solution[position_1], enhanced_solution[(position_1+1)%len(enhanced_solution)]] + cost_matrix[enhanced_solution[position_2], enhanced_solution[(position_2+1)%len(enhanced_solution)]] )
            )

In [23]:
import time

In [51]:
# @jit()
def enhance_solution(initial_solution, closest_neighbours, cost_matrix):

    enhanced_solution = list(initial_solution)
    previous_solution = 0
    current_solution = calculate_performance(initial_solution, cost_matrix)
    total_time = 0
    while True:
        start = time.time()
        previous_solution = current_solution
        best_delta = 0                  # e.g. -10
        best_edges_positions = None     # e.g. [1, 10]
        best_type = None                # before or after
        for position in range(len(initial_solution)):
            for neighbour in closest_neighbours[enhanced_solution[position]]:
                position_neighbour = enhanced_solution.index(neighbour)
                if abs(position - position_neighbour) == 1: continue
                    
                
                for swap_type in ('before', 'after'):
                    if swap_type == 'before':
                        delta = calculate_change_before(enhanced_solution, cost_matrix, position_1=position, position_2=position_neighbour)
                        
                    elif swap_type == 'after':
                        delta = calculate_change_after(enhanced_solution, cost_matrix, position_1=position, position_2=position_neighbour)
                        
                    
                    if delta < best_delta:
                        best_delta = delta
                        best_edges_positions = [position, position_neighbour]
                        best_type = swap_type
                
                

        end = time.time()
        total_time += end - start     
                        
        
        # break
        if best_delta == 0:
            return enhanced_solution, total_time
        else:
            
            if best_type == 'before':
                if best_edges_positions[0] < best_edges_positions[1]:
                    enhanced_solution = enhanced_solution[:best_edges_positions[0]] + enhanced_solution[best_edges_positions[0]:best_edges_positions[1]][::-1] + enhanced_solution[best_edges_positions[1]:]
                else:
                    enhanced_solution = enhanced_solution[best_edges_positions[1]:best_edges_positions[0]] + list(enhanced_solution[best_edges_positions[0]:] + enhanced_solution[:best_edges_positions[1]])[::-1]
            elif best_type == 'after':
                if best_edges_positions[0] < best_edges_positions[1]:
                    # print(enhanced_solution)
                    enhanced_solution = enhanced_solution[:(best_edges_positions[0]+1)%len(enhanced_solution)] + enhanced_solution[(best_edges_positions[0]+1)%len(enhanced_solution):(best_edges_positions[1]+1)][::-1] + enhanced_solution[(best_edges_positions[1]+1):]
                    # print('-------------------------')
                    # print(enhanced_solution, best_edges_positions, calculate_performance(enhanced_solution, cost_matrix) - previous_solution, best_delta)
                    # print('\n')
                    # print("XDASDQWEDASD")
                else:
                    # print(enhanced_solution)
                    enhanced_solution = enhanced_solution[best_edges_positions[1]+1:best_edges_positions[0]+1] + list(enhanced_solution[(best_edges_positions[0]+1):] + enhanced_solution[:best_edges_positions[1]+1])[::-1]
                    # print('-------------------------')
                    # print(enhanced_solution, best_edges_positions, calculate_performance(enhanced_solution, cost_matrix) - previous_solution, best_delta)
                    # print('\n')
                    # enhanced_solution[:(best_edges_positions[0]+1)%len(enhanced_solution)] + enhanced_solution[(best_edges_positions[0]+1)%len(enhanced_solution):(best_edges_positions[1]+1)][::-1] + enhanced_solution[(best_edges_positions[1]+1):]
                    
                    # time.sleep(15)
                # print('WTF')
                # if best_edges_positions[0] < best_edges_positions[1]:
                    
                #     
                # else:

            
                
        current_solution = calculate_performance(enhanced_solution, cost_matrix)
        assert current_solution < previous_solution
        
    return 0, 1


In [52]:
from tqdm import tqdm

In [55]:
def start_enhancement(starting_solution_type: str, cost_matrix, distance_matrix):
    total_cost = 0
    total_cost_random = 0
    total_time = 0
    for i in tqdm(range(len(distance_matrix))):
        # print(i)
        if starting_solution_type == 'greedy':
            initial_solution = greedy_cycle(cost_matrix, i, limit=100)
        elif starting_solution_type == 'random':
            initial_solution = random_solution(cost_matrix, limit=100)
        else:
            return 'Bad type'
        initial_cost = calculate_performance(initial_solution, cost_matrix)
        total_cost_random += initial_cost

        closest_neighbours = calculate_closest_neighbours_solution(distance_matrix, initial_solution, n_neighbours=10)
        return closest_neighbours
        # enhanced_solution, time_spent = enhance_solution(initial_solution, closest_neighbours, distance_matrix)
        # total_cost += calculate_performance(enhanced_solution, cost_matrix)
        # total_time += time_spent
    return total_cost / len(cost_matrix), total_cost_random / len(cost_matrix), total_time, total_time / len(cost_matrix)
        
        

        


In [56]:
start_enhancement('random', cost_matrix=cost_matrix_C, distance_matrix=dist_matrix_C)

  0%|          | 0/200 [00:00<?, ?it/s]


{32: [62, 93, 81, 7, 15, 53, 18, 47, 22, 28],
 74: [47, 61, 30, 25, 93, 7, 85, 62, 53, 18],
 75: [1, 83, 17, 97, 41, 39, 90, 16, 9, 60],
 80: [8, 31, 95, 14, 73, 89, 13, 42, 12, 26],
 29: [2, 4, 77, 43, 91, 50, 86, 97, 83, 39],
 93: [62, 7, 47, 32, 53, 18, 15, 74, 22, 30],
 60: [9, 17, 99, 83, 97, 90, 75, 45, 88, 51],
 28: [15, 22, 18, 53, 55, 54, 30, 21, 47, 62],
 68: [66, 24, 98, 6, 78, 45, 5, 72, 51, 67],
 1: [41, 75, 39, 97, 83, 17, 16, 11, 90, 92],
 83: [97, 17, 75, 39, 9, 60, 1, 41, 29, 90],
 90: [99, 17, 9, 60, 33, 75, 51, 83, 92, 26],
 65: [3, 40, 34, 58, 84, 37, 49, 0, 38, 36],
 85: [25, 57, 23, 10, 37, 84, 64, 82, 36, 74],
 30: [55, 22, 47, 18, 53, 54, 36, 74, 15, 28],
 64: [56, 82, 20, 71, 25, 57, 85, 46, 61, 23],
 35: [86, 69, 19, 50, 0, 43, 38, 58, 49, 91],
 77: [4, 2, 29, 43, 50, 86, 91, 35, 69, 38],
 18: [53, 22, 15, 28, 55, 47, 30, 62, 93, 54],
 76: [49, 91, 54, 38, 0, 50, 55, 28, 69, 36],
 6: [66, 98, 68, 24, 72, 78, 67, 87, 45, 94],
 46: [63, 82, 70, 96, 59, 57, 23, 6

In [50]:
start_enhancement('random', cost_matrix=cost_matrix_D, distance_matrix=dist_matrix_D)

100%|██████████| 200/200 [00:55<00:00,  3.61it/s]


(75083.995, 218860.99, 0.04975008964538574, 0.0002487504482269287)